In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_two_year'].values

In [4]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6466923632859551, 0.6422404239942183)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [10]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.0025, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge18',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_charges1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdemeanor1',
 'p_misdemeanor3',
 'p_property1',
 'p_property2',
 'p_drug1',
 'p_trespass1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'one_year1']

In [11]:
c_grid={'C': [0.001, 0.002, 0.0025]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [12]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.0025}, {'C': 0.0025}, {'C': 0.0025}, {'C': 0.0025}, {'C': 0.0025}],
 0.6633026579940577,
 0.0008579233080876714)

### RiskSLIM

In [13]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

16

In [14]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [15]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'property_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 07:33 PM | 213 rows in lookup table
02/15/20 @ 07:33 PM | ------------------------------------------------------------
02/15/20 @ 07:33 PM | runnning initialization procedure
02/15/20 @ 07:33 PM | ------------------------------------------------------------
02/15/20 @ 07:33 PM | CPA produced 2 cuts
02/15/20 @ 07:33 PM | running naive rounding on 5 solutions
02/15/20 @ 07:33 PM | best objective value: 0.5287
02/15/20 @ 07:33 PM | rounding produced 5 integer solutions
02/15/20 @ 07:33 PM | best objective value is 0.5121
02/15/20 @ 07:33 PM | running sequential rounding on 5 solutions
02/15/20 @ 07:33 PM | best objective value: 0.5287
02/15/20 @ 07:33 PM | sequential rounding produced 3 integer solutions
02/15/20 @ 07:33 PM | best objective value: 0.3570
02/15/20 @ 07:33 PM | polishing 8 solutions
02/15/20 @ 07:33 PM | best objective value: 0.3570
02/15/20 @ 07:33 PM | polishing produced 4 integer solutions
02/15/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:33 PM | adding 270 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1579.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 159458 40891        0.1496    12        0.1536        0.1496   925257    2.63%           rho_0 U 159458 123848     25
Elapsed time = 37.36 sec. (41255.88 ticks, tree = 11.99 MB, solutions = 7)
 163248 41712        0.1520    10        0.1536        0.1496   937125    2.62%           rho_1 D 163248 163246     21
 166723 42359        cutoff              0.1536        0.1496   951748    2.59%           rho_8 U 166723 166713     34
 170080 42840        0.1530    14        0.1536        0.1497   966715    2.56%           rho_2 U 170080 107915     26
 173619 42967        0.1529    13        0.1536        0.1497   980499    2.53%           rho_3 U 173619 173617     27
 176850 43130        0.1525    10        0.1536        0.1498   995331    2.50%           rho_4 D 176850 176849     35
 180268 43388        0.1499    10        0.1536        0.1498  1009454    2.48%           rho_8 U 180268 180267     33
 183783 43940        0.1511    11        0.1536        0.1498  1022952    2.45%           rh

 510698 22597        0.1528    10        0.1536        0.1528  2243853    0.51%           rho_8 D 510698 510696     38
 522893 16073        0.1530    15        0.1536        0.1530  2282002    0.40%           rho_4 U 522893 522892     27
 535136  9875        cutoff              0.1536        0.1532  2316059    0.26%           rho_1 U 535136 487418     25
 547447  2638        0.1535     7        0.1536        0.1535  2345002    0.09%         alpha_7 U 547447 287256     45

User cuts applied:  533

Root node processing (before b&c):
  Real time             =    0.08 sec. (1.62 ticks)
Sequential b&c:
  Real time             =  108.52 sec. (157834.32 ticks)
                          ------------
Total (root+branch&cut) =  108.59 sec. (157835.94 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 07:35 PM | 213 rows in lookup table
02/15/20 @ 07:35 PM | ------------------------------------------------------------
02/15/20 @ 07:35 PM | runnning initialization procedu

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:35 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1555.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:29 PM | adding 256 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1611.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 178337 36532        0.1511    10        0.1535        0.1500   811940    2.26%           rho_3 U 178337 114164     45
 182088 36833        0.1530     6        0.1535        0.1500   827094    2.23%           rho_1 N 182088 182086     42
Elapsed time = 25.56 sec. (41257.26 ticks, tree = 10.77 MB, solutions = 9)
 185966 37388        0.1527    11        0.1535        0.1501   840374    2.20%        alpha_11 D 185966 185965     47
 189811 38007        cutoff              0.1535        0.1501   853765    2.18%        alpha_11 U 189811 180955     43
 193470 37847        cutoff              0.1535        0.1502   866816    2.15%           rho_3 D 193470 145770     24
 197030 37814        0.1530     3        0.1535        0.1502   879913    2.12%         alpha_9 D 197030 197029     43
 200535 37826        cutoff              0.1535        0.1502   893678    2.10%           rho_5 N 200535 200534     42
 204216 38154        0.1524     7        0.1535        0.1503   907336    2.08%         alph

02/15/20 @ 08:31 PM | 213 rows in lookup table
02/15/20 @ 08:31 PM | ------------------------------------------------------------
02/15/20 @ 08:31 PM | runnning initialization procedure
02/15/20 @ 08:31 PM | ------------------------------------------------------------
02/15/20 @ 08:31 PM | CPA produced 2 cuts
02/15/20 @ 08:31 PM | running naive rounding on 6 solutions
02/15/20 @ 08:31 PM | best objective value: 0.4973
02/15/20 @ 08:31 PM | rounding produced 5 integer solutions
02/15/20 @ 08:31 PM | best objective value is 0.5557
02/15/20 @ 08:31 PM | running sequential rounding on 6 solutions
02/15/20 @ 08:31 PM | best objective value: 0.4973
02/15/20 @ 08:31 PM | sequential rounding produced 2 integer solutions
02/15/20 @ 08:31 PM | best objective value: 0.3569
02/15/20 @ 08:31 PM | polishing 7 solutions
02/15/20 @ 08:31 PM | best objective value: 0.3569
02/15/20 @ 08:31 PM | polishing produced 5 integer solutions
02/15/20 @ 08:31 PM | initialization produced 11 feasible solutions
02/

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:31 PM | adding 262 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1532.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 166669 31890        cutoff              0.1525        0.1497   853479    1.85%           rho_4 U 166669  70847     20
 170027 32118        0.1512     7        0.1525        0.1497   866560    1.82%           rho_5 D 170027 170026     34
 173391 32436        cutoff              0.1525        0.1498   878461    1.79%           rho_3 U 173391  84996     20
 176784 32630        0.1502    11        0.1525        0.1498   890735    1.76%          rho_14 D 176784 176783     26
 180086 32774        cutoff              0.1525        0.1498   903477    1.73%          rho_12 U 180086  90016     26
 183585 33276        0.1499    11        0.1525        0.1499   915372    1.72%           rho_8 U 183585 183583     48
 186932 33156        cutoff              0.1525        0.1499   928163    1.70%           rho_5 D 186932 186931     32
 190297 33241        0.1514     8        0.1525        0.1499   939729    1.68%           rho_1 U 190297 190295     31
 193719 33508        cutoff              0.1525 

In [16]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.7508343479658288, 0.6117343611981049)

In [17]:
#### save results
summary_property2_ky_inter_model = {"cart": cart_summary,
                                    "ebm": ebm_summary,
                                    "stumps": stumps_summary,
                                    "riskslim": riskslim_summary}

In [18]:
path = "./results/interpretable/two-year/"
results = [["Property",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)